In [3]:
# !pip install selenium

In [1]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException
import time
import re

In [5]:
def create_driver():
    service = Service()
    options = webdriver.ChromeOptions()
    driver = webdriver.Chrome(service=service, options=options)
    return driver

In [5]:
def open_browser(url):
    driver = create_driver()
    driver.get(url)
    driver.implicitly_wait(2)
    driver.maximize_window()
    return driver

In [6]:
def perform_page_down(scroll_element, num_pages=50, delay=1):
    for _ in range(num_pages):
        scroll_element.send_keys(Keys.PAGE_DOWN)
        time.sleep(delay)

In [9]:
LOC = '서울'
food_lst = ['간장게장', '갈비찜', '갈비탕', '감자전', '감자탕', '곱창구이', '김밥', '김치전', '김치찌개', '닭갈비', '닭볶음탕', '도토리묵', '된장찌개', '떡볶이', '막국수', '물냉면',
 '물회', '미역국', '배추김치', '불고기', '비빔냉면', '비빔밥', '삼겹살', '삼계탕', '설렁탕', '순대', '순두부찌개', '양념게장', '양념치킨', '육회', '잡채', '제육볶음',
 '족발', '주꾸미볶음', '짜장면', '칼국수', '파전', '해물찜', '황태구이', '후라이드치킨']
crawl_lst = [f'{LOC} {food}' for food in food_lst]
print(crawl_lst)
print(len(crawl_lst))

['서울 간장게장', '서울 갈비찜', '서울 갈비탕', '서울 감자전', '서울 감자탕', '서울 곱창구이', '서울 김밥', '서울 김치전', '서울 김치찌개', '서울 닭갈비', '서울 닭볶음탕', '서울 도토리묵', '서울 된장찌개', '서울 떡볶이', '서울 막국수', '서울 물냉면', '서울 물회', '서울 미역국', '서울 배추김치', '서울 불고기', '서울 비빔냉면', '서울 비빔밥', '서울 삼겹살', '서울 삼계탕', '서울 설렁탕', '서울 순대', '서울 순두부찌개', '서울 양념게장', '서울 양념치킨', '서울 육회', '서울 잡채', '서울 제육볶음', '서울 족발', '서울 주꾸미볶음', '서울 짜장면', '서울 칼국수', '서울 파전', '서울 해물찜', '서울 황태구이', '서울 후라이드치킨']
40


In [ ]:
food = []
store = []
addr = []
tel = []
rank1 = []
rank2 = []
rank3 = []
rank4 = []
rank5 = []
visitor_review = []

In [ ]:
for crawl in crawl_list:
    search_food = crawl
    driver = open_browser('https://map.naver.com/')
    search_element = (By.CLASS_NAME, 'input_search')
    search_box = WebDriverWait(driver, 5).until(
        EC.presence_of_element_located(search_element))
    
    # 키 입력 및 엔터
    search_box.click()
    time.sleep(1)
    search_box.clear()
    search_box.send_keys(search_food)
    search_box.send_keys(Keys.ENTER)
    time.sleep(2)
    
    # searchIframe 진입
    searchIF = driver.find_element(By.CSS_SELECTOR, '#searchIframe')
    driver.switch_to.frame(searchIF)
    
    # searchIframe 스크롤
    searchIF_scroll_locator = (By.CSS_SELECTOR, 'body')
    searchIF_scroll = WebDriverWait(driver, 5).until(
        EC.element_to_be_clickable(searchIF_scroll_locator))
    
    searchIF_scroll.click()
    time.sleep(1)
    
    perform_page_down(searchIF_scroll, 35)

    li_elements = driver.find_element(By.CSS_SELECTOR, '#_pcmap_list_scroll_container').find_elements(By.TAG_NAME, "li")
    time.sleep(1)
                
    for i in range(len(li_elements)):
        if i == 0:
            element1 = li_elements[i+1].find_element(By.CLASS_NAME, 'TYaxT')
            driver.execute_script("arguments[0].click();", element1)
            time.sleep(1)
    
        # element0 = li_elements[i].find_element(By.CLASS_NAME, 'YwYLL')
        element0 = li_elements[i].find_element(By.CLASS_NAME, 'TYaxT')
        driver.execute_script("arguments[0].click();", element0)
        time.sleep(1)
        
        driver.switch_to.default_content()
        time.sleep(2)
            
        ## entryIframe
        WebDriverWait(driver, 10).until(
            EC.frame_to_be_available_and_switch_to_it((By.ID, 'entryIframe')))
        # driver.switch_to.frame(entryIF_element)
    
        # entryIframe 클릭
        entry_click_element = (By.CSS_SELECTOR, '#app-root > div > div > div > div.place_fixed_maintab > div > div > div > div > a:nth-child(1) > span')
        entry_click = WebDriverWait(driver, 10).until(
            EC.element_to_be_clickable(entry_click_element)
        )
        entry_click.click()
        time.sleep(1) 
        
        entryIF_scroll_locator = (By.CSS_SELECTOR, 'body')
        entryIF_scroll = WebDriverWait(driver, 5).until(
            EC.presence_of_element_located(entryIF_scroll_locator))  
            
        perform_page_down(entryIF_scroll, 6)
            
        # data crawling
        food.append(search_food.split(' ')[1])
    
        name_element = (By.CSS_SELECTOR, '#_title > div > span.Fc1rA') 
        name = WebDriverWait(driver, 10).until(
            EC.presence_of_element_located(name_element)).text
        store.append(name)
    
        reviews = []  # reviews 변수 초기화
        try:
            reviews_element = (By.CLASS_NAME, 'nbD78')
            reviews = WebDriverWait(driver, 5).until(
                EC.presence_of_all_elements_located(reviews_element)
            )
        except TimeoutException:
            for i in range(1, 6):
                # 기존에 생성되어 있는 변수에 append
                globals()[f'rank{i}'].append('미등록')
                    
        for index, review in enumerate(reviews):
            # 리뷰의 텍스트 가져오기
            review_text = review.text.replace('\n', '').replace('이 키워드를 선택한 인원', '').replace('"', '')
            modified_string = re.sub(r'(\D)(\d+)(\D|$)', r'\1 \2\3', review_text)
    
            if index == 0:
                rank1.append(modified_string)
            elif index == 1:
                rank2.append(modified_string)
            elif index == 2:
                rank3.append(modified_string)
            elif index == 3:
                rank4.append(modified_string)
            elif index == 4:
                rank5.append(modified_string)
                        
        # 방문자 리뷰 더보기 클릭 
        try:
            more_text_element = (By.CSS_SELECTOR, 'div.RGkHL > a > span.rvCSr > span')
            more_text = WebDriverWait(driver, 5).until(
                EC.presence_of_element_located(more_text_element))
            driver.execute_script("arguments[0].click();", more_text)
        except TimeoutException:
            pass  
            
        # 방문자 리뷰 저장
        try:
            vreview_element = (By.CSS_SELECTOR, 'div.TraH1 > ul > li:nth-child(1) > div.IEbo1 > div.GP2eR > div.RGkHL > a > span')
            vreview = WebDriverWait(driver, 5).until(
                EC.presence_of_element_located(vreview_element)).text.replace('\n', '')
        except TimeoutException:
            vreview = '미등록'
            
        visitor_review.append(vreview)
    
        # 주소 저장
        try:
            addr_element = (By.CLASS_NAME, 'LDgIH')
            addr_lo = WebDriverWait(driver, 5).until(
                EC.presence_of_element_located(addr_element)).text
        except TimeoutException:
            addr_lo = '미등록'
            
        addr.append(addr_lo)
        # 전화번호 저장   
        try:
            tel_element = (By.CLASS_NAME, 'xlx7Q')
            tel_lo = WebDriverWait(driver, 5).until(
                EC.presence_of_element_located(tel_element)).text
            
        except TimeoutException:
            tel_lo = '미등록'
        tel.append(tel_lo)
        
        driver.switch_to.default_content()
        time.sleep(1)
        driver.switch_to.frame(searchIF)
        
driver.close()

In [ ]:
data = {
    'food': food,
    'store_name': store,
    'addr': addr,
    'tel': tel,
    're_rank1': rank1,
    're_rank2': rank2,
    're_rank3': rank3,
    're_rank4': rank4,
    're_rank5': rank5,
    're_visitor': visitor_review
}

# 데이터프레임 생성
df = pd.DataFrame(data)

# 결과 출력
df

df.to_csv('/data/restaurant_list.csv', index=False)

In [2]:
restaurant_list = pd.read_csv('./data/restaurant_list.csv')
restaurant_list.head()

,food,store_name,addr,tel,re_rank1,re_rank2,re_rank3,re_rank4,re_rank5,re_visitor
0,간장게장,춘남식당,서울 마포구 성미산로 97 춘남식당,0507-1384-0693,음식이 맛있어요 35,재료가 신선해요 28,양이 많아요 24,친절해요 23,특별한 메뉴가 있어요 12,"꽃게탕 소자에 칼국수사리, 볶음밥을 추가해서 먹었어요! 연평도산 꽃게라 그런지 야들..."
1,간장게장,장안동 먹깨비 간장게장 해물아구찜,서울 동대문구 장한로 92 1층,0507-1393-2158,음식이 맛있어요 456,친절해요 225,재료가 신선해요 202,양이 많아요 164,매장이 넓어요 115,남친 부모님이 매운걸 못드셔서 안맵게로 부탁 존나 맵게해야 더 맛있을듯 이모님 사장...
2,간장게장,예담밥상,서울 광진구 동일로22길 13,0507-1443-0073,음식이 맛있어요 96,재료가 신선해요 52,친절해요 42,비싼 만큼 가치있어요 33,차분한 분위기에요 30,🦀간장게장과 갈비가 같이 나오는 특선을 추천 해주셔서 먹었습니다~ ^^ 확실히 찬들...
3,간장게장,명현만간장게장무한리필,서울 마포구 양화로 45 메세나폴리스 123호~129호,0507-1302-3837,음식이 맛있어요 677,재료가 신선해요 385,친절해요 370,양이 많아요 328,가성비가 좋아요 315,명현만의 간장게장 합정점! 평소 명현만 선수의 팬이었다면 더욱 좋을 것 같아요! 가...
4,간장게장,진미식당,서울 마포구 마포대로 186-6,02-3211-4468,음식이 맛있어요 364,재료가 신선해요 180,특별한 메뉴가 있어요 105,친절해요 73,특별한 날 가기 좋아요 41,맛있어요~예약이 힘들었는데 다행히 성공했어요ㅋ 들어가서 앉고 거의바로 음식이나왔어요...


In [3]:
from hdfs import InsecureClient # hadoop 서버와 연결하는  client객체 모듈(hdfs 접근)
import requests # web 사용
import json # json 기능 사용
import datetime as dt

In [4]:
## hdfs client 연결객체 생성
client=InsecureClient('http://localhost:9870',user='lab11')

In [5]:
cols=['food','store_name','addr','tel', 're_rank1', 're_rank2', 're_rank3', 're_rank4', 're_rank5', 're_visitor']
data=[]      

In [6]:
for idx in restaurant_list.index : 
    tuple_t = []
    tmp = restaurant_list.loc[idx]
    tuple_t.append(str(tmp.food))
    tuple_t.append(str(tmp.store_name))
    tuple_t.append(str(tmp.addr))
    tuple_t.append(str(tmp.tel))
    tuple_t.append(str(tmp.re_rank1))
    tuple_t.append(str(tmp.re_rank2))
    tuple_t.append(str(tmp.re_rank3))
    tuple_t.append(str(tmp.re_rank4))
    tuple_t.append(str(tmp.re_rank5))
    tuple_t.append(str(tmp.re_visitor))
    data.append(dict(zip(cols,tuple_t)))

In [7]:
data

[{'food': '간장게장',
  'store_name': '춘남식당',
  'addr': '서울 마포구 성미산로 97 춘남식당',
  'tel': '0507-1384-0693',
  're_rank1': '음식이 맛있어요 35',
  're_rank2': '재료가 신선해요 28',
  're_rank3': '양이 많아요 24',
  're_rank4': '친절해요 23',
  're_rank5': '특별한 메뉴가 있어요 12',
  're_visitor': '꽃게탕 소자에 칼국수사리, 볶음밥을 추가해서 먹었어요! 연평도산 꽃게라 그런지 야들야들하고 부드러운 살이 꽉 차있더라구요❤️소 사이즈인데도 꽃게가 커서 양이 매우 많았어요!매운탕 국물은 칼칼하고 얼큰해서 요즘같이 추운 날씨에 딱 잘 어울려요>_< 밥이랑 게살이랑 비벼먹으면 밥 한 공기 뚝딱하는 밥도둑!국물이 워낙 맛있어서 칼국수랑 볶음밥도 맛도리였습니다!! 이정도 퀄리티의 꽃게탕을 이렇게 저렴한 가격대에 맛볼 수 있다니….❤️이번 겨울에 제 최애 단골집이 될 것 같습니다❤️'},
 {'food': '간장게장',
  'store_name': '장안동 먹깨비 간장게장 해물아구찜',
  'addr': '서울 동대문구 장한로 92 1층',
  'tel': '0507-1393-2158',
  're_rank1': '음식이 맛있어요 456',
  're_rank2': '친절해요 225',
  're_rank3': '재료가 신선해요 202',
  're_rank4': '양이 많아요 164',
  're_rank5': '매장이 넓어요 115',
  're_visitor': '남친 부모님이 매운걸 못드셔서 안맵게로 부탁 존나 맵게해야 더 맛있을듯 이모님 사장님 다 친절하심 잘 먹었어요ㅋㅎ'},
 {'food': '간장게장',
  'store_name': '예담밥상',
  'addr': '서울 광진구 동일로22길 13',
  'tel': '0507-1443-0073',
  're_rank1': '음식이 맛있어요 96',

In [9]:
# json에 포함시킬 meta data 구성
res = {
        'meta':{
            'desc':'음식점 리뷰 크롤링 데이터',
            'cols':{
                'food':'food'
                ,'store_name':'store_name'
                ,'addr':'addr' 
                ,'tel' : 'tel'
                ,'re_rank1' : 're_rank1'
                ,'re_rank2' : 're_rank2'
                ,'re_rank3' : 're_rank3'
                ,'re_rank4' : 're_rank4'
                ,'re_rank5' : 're_rank5'
                ,'re_visitor' : 're_visitor'
                  },
        },
        'data':data
}
res

{'meta': {'desc': '음식점 리뷰 크롤링 데이터',
  'cols': {'food': 'food',
   'store_name': 'store_name',
   'addr': 'addr',
   'tel': 'tel',
   're_rank1': 're_rank1',
   're_rank2': 're_rank2',
   're_rank3': 're_rank3',
   're_rank4': 're_rank4',
   're_rank5': 're_rank5',
   're_visitor': 're_visitor'}},
 'data': [{'food': '간장게장',
   'store_name': '춘남식당',
   'addr': '서울 마포구 성미산로 97 춘남식당',
   'tel': '0507-1384-0693',
   're_rank1': '음식이 맛있어요 35',
   're_rank2': '재료가 신선해요 28',
   're_rank3': '양이 많아요 24',
   're_rank4': '친절해요 23',
   're_rank5': '특별한 메뉴가 있어요 12',
   're_visitor': '꽃게탕 소자에 칼국수사리, 볶음밥을 추가해서 먹었어요! 연평도산 꽃게라 그런지 야들야들하고 부드러운 살이 꽉 차있더라구요❤️소 사이즈인데도 꽃게가 커서 양이 매우 많았어요!매운탕 국물은 칼칼하고 얼큰해서 요즘같이 추운 날씨에 딱 잘 어울려요>_< 밥이랑 게살이랑 비벼먹으면 밥 한 공기 뚝딱하는 밥도둑!국물이 워낙 맛있어서 칼국수랑 볶음밥도 맛도리였습니다!! 이정도 퀄리티의 꽃게탕을 이렇게 저렴한 가격대에 맛볼 수 있다니….❤️이번 겨울에 제 최애 단골집이 될 것 같습니다❤️'},
  {'food': '간장게장',
   'store_name': '장안동 먹깨비 간장게장 해물아구찜',
   'addr': '서울 동대문구 장한로 92 1층',
   'tel': '0507-1393-2158',
   're_rank1': '음식이 맛있어요 456',
   

In [10]:
file_dir='/data/'
file_name='restaurant_list.json'
client.write(file_dir+file_name,json.dumps(res,ensure_ascii=False),encoding='utf=8')